# 컨텍스추얼 압축
- 검색의 한 가지 문제는 데이터 저장 시스템에 데이터를 입력할 때 특정 쿼리를 알지 못한다는 점입니다.
- 이로 인해 쿼리와 가장 관련 있는 정보가 많은 불필요한 텍스트가 포함된 문서에 묻힐 수 있습니다.
- 전체 문서를 애플리케이션에 전달하면 더 비싼 LLM 호출과 더 나쁜 응답을 초래할 수 있습니다.
 
# 컨텍스추얼 압축은 이를 해결하기 위한 것입니다.
- 아이디어는 간단합니다: 검색된 문서를 그대로 반환하는 대신 주어진 쿼리의 컨텍스트를 사용하여 압축할 수 있습니다.
- 여기서 "압축"은 개별 문서의 내용을 압축하고 문서를 전체적으로 필터링하는 것을 의미합니다.
 
# 컨텍스추얼 압축 검색기를 사용하려면 다음이 필요합니다:
- 기본 검색기
- 문서 압축기
- 컨텍스추얼 압축 검색기는 쿼리를 기본 검색기에 전달하고 초기 문서를 받아 문서 압축기에 전달합니다.
- 문서 압축기는 문서 목록을 받아 문서의 내용을 줄이거나 문서를 완전히 삭제하여 목록을 단축합니다.

In [7]:
# Helper function for printing docs


def pretty_print_docs(docs):
    print(
        f"\n{'-' * 100}\n".join(
            [f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]
        )
    )

 # 기본 벡터 스토어 검색기 사용
- 간단한 벡터 스토어 검색기를 초기화하고 2023년 국정 연설을 (청크로) 저장해보겠습니다.
- 예시 질문을 주면 검색기가 한두 개의 관련 문서와 몇 개의 관련 없는 문서를 반환하는 것을 볼 수 있습니다.
- 그리고 관련 문서에도 많은 관련 없는 정보가 포함되어 있습니다.


In [8]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from dotenv import load_dotenv
load_dotenv('../dot.env')

query = "테마지도는?"

documents = TextLoader("./files/1train-geon-Manual-simple.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
retriever = FAISS.from_documents(texts, OpenAIEmbeddings()).as_retriever()

docs = retriever.invoke(query)
pretty_print_docs(docs)

Document 1:

2.2. 표준/편집 테마
지도 빌더를 통해 제작된 지도를 표준, 편집 테마를 활용하여 앱의 레이아웃 및 스타일을 설정 하고 위젯들을 선택하여 새로운 앱을 생성할 수 있다.

2.2.1. 앱 만들기 화면
 테마 목록을 조회하고 테마를 선택하여 앱 편집 화면으로 이동한다.

2.2.2. 메인 설정
 메인 패널에서 사용할 사용자 지도를 선택한다. 

2.2.3. 레이아웃 설정
 타이틀 및 제어도구 위치를 설정한다.

2.2.4. 테마 설정
 헤더 및 푸터의 색상 및 타이틀 등 테마를 설정한다.

2.2.5. 위젯 설정
 웹앱의 위젯을 추가하고 저장한다.
 
헤더 영역 위젯 
앱 기본정보 
웹앱에 추가된 지도에 대한 상세정보를 확인할 수 있는 위젯
레이어 영역 위젯
 레이어 검색 레이어를 검색하고 추가할 수 있는 위젯
 레이어 스타일 레이어의 색상 및 라벨 등을 설정할 수 있는 위젯
 속성 레이어의 속성정보를 확인 할 수 있는 위젯
 속성필터 레이어의 속성으로 필터를 적용할 수 있는 위젯

상단 영역 위젯
 북마크 현재 지도의 위치를 저장할 수 있는 위젯
 오버뷰 오버뷰를 표출하는 위젯
 주소 검색 주소를 검색할 수 있는 위젯
 행정경계 표시 현재 지도의 행정경계 위치를 표시하고 다른 행정경계로 이동할 수 있는 위젯
 저장 지도를 PNG 또는 PDF로 저장할 수 있는 위젯
 인쇄 지도를 프린트 할 수 있는 위젯
 피쳐속성폼 위젯을 클릭한 후 지도상에 피쳐를 클릭하면 피쳐정보를 수정 할 수 있는 위젯
----------------------------------------------------------------------------------------------------
Document 2:

﻿맵픽이란
Mappick은 누구나 손쉽게 공간정보가 포함된 지도를 제작하여 업무에 적용 할 수 있는 SaaS(Software as a Service) GIS 서비스입니다. Mappick을 사용하면 인터넷을 통해 공개된 공공 데이터나, 본인이

- 컨텍스추얼 압축을 LLMChainExtractor와 함께 추가
- 기본 검색기를 ContextualCompressionRetriever로 감쌉니다.
- LLMChainExtractor를 추가하여 초기 반환된 문서를 반복하고 각 문서에서 쿼리와 관련된 내용만 추출합니다.

In [9]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain_openai import OpenAI

llm = OpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    query
)
pretty_print_docs(compressed_docs)

Document 1:

2.2. 표준/편집 테마
지도 빌더를 통해 제작된 지도를 표준, 편집 테마를 활용하여 앱의 레이아웃 및 스타일을 설정 하고 위젯들을 선택하여 새로운 앱을 생성할 수 있다.

2.2.1. 앱 만들기 화면
 테마 목록을 조회하고 테마를 선택하여 앱 편집 화면으로 이동한다.

2.2.2. 메인 설정
 메인 패널에서 사용할 사용자 지도를 선택한다. 

2.2.3. 레이아웃 설정
 타이틀 및 제어도구 위치를 설정한다.

2.2.4. 테마 설정
 헤더 및 푸터의 색상 및 타이틀 등 테마를 설정
----------------------------------------------------------------------------------------------------
Document 2:

테마지도를 만들어 활용할 수 있고 서로 공유할 수 있습니다.
----------------------------------------------------------------------------------------------------
Document 3:

행정구역
----------------------------------------------------------------------------------------------------
Document 4:

- 지도빌더) 어떤 종류의 레이어를 지도에 추가할 수 있습니까?
- Shape File(ZIP), 주소, PNU, X/Y좌표가 포함된 엑셀/CSV/TXT 파일, OGC 표준의 WFS/WMS/WMTS 웹레이어 URL, DXF 캐드파일 등


- 더 많은 내장 압축기: 필터
- LLMChainFilter
- LLMChainFilter는 문서 내용을 조작하지 않고 처음 검색된 문서 중 어떤 것을 필터링하고 반환할지 결정하는 LLM 체인을 사용하는 더 간단하지만 더 강력한 압축기입니다.

In [10]:
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    query
)
pretty_print_docs(compressed_docs)

Document 1:

2.2. 표준/편집 테마
지도 빌더를 통해 제작된 지도를 표준, 편집 테마를 활용하여 앱의 레이아웃 및 스타일을 설정 하고 위젯들을 선택하여 새로운 앱을 생성할 수 있다.

2.2.1. 앱 만들기 화면
 테마 목록을 조회하고 테마를 선택하여 앱 편집 화면으로 이동한다.

2.2.2. 메인 설정
 메인 패널에서 사용할 사용자 지도를 선택한다. 

2.2.3. 레이아웃 설정
 타이틀 및 제어도구 위치를 설정한다.

2.2.4. 테마 설정
 헤더 및 푸터의 색상 및 타이틀 등 테마를 설정한다.

2.2.5. 위젯 설정
 웹앱의 위젯을 추가하고 저장한다.
 
헤더 영역 위젯 
앱 기본정보 
웹앱에 추가된 지도에 대한 상세정보를 확인할 수 있는 위젯
레이어 영역 위젯
 레이어 검색 레이어를 검색하고 추가할 수 있는 위젯
 레이어 스타일 레이어의 색상 및 라벨 등을 설정할 수 있는 위젯
 속성 레이어의 속성정보를 확인 할 수 있는 위젯
 속성필터 레이어의 속성으로 필터를 적용할 수 있는 위젯

상단 영역 위젯
 북마크 현재 지도의 위치를 저장할 수 있는 위젯
 오버뷰 오버뷰를 표출하는 위젯
 주소 검색 주소를 검색할 수 있는 위젯
 행정경계 표시 현재 지도의 행정경계 위치를 표시하고 다른 행정경계로 이동할 수 있는 위젯
 저장 지도를 PNG 또는 PDF로 저장할 수 있는 위젯
 인쇄 지도를 프린트 할 수 있는 위젯
 피쳐속성폼 위젯을 클릭한 후 지도상에 피쳐를 클릭하면 피쳐정보를 수정 할 수 있는 위젯
----------------------------------------------------------------------------------------------------
Document 2:

﻿맵픽이란
Mappick은 누구나 손쉽게 공간정보가 포함된 지도를 제작하여 업무에 적용 할 수 있는 SaaS(Software as a Service) GIS 서비스입니다. Mappick을 사용하면 인터넷을 통해 공개된 공공 데이터나, 본인이

- EmbeddingsFilter
- 각 검색된 문서에 대해 추가 LLM 호출을 하는 것은 비용이 많이 들고 느립니다.
- EmbeddingsFilter는 문서와 쿼리를 임베딩하여 쿼리와 충분히 유사한 임베딩을 가진 문서만 반환함으로써 더 저렴하고 빠른 옵션을 제공합니다.


In [11]:
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()
embeddings_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    query
)
pretty_print_docs(compressed_docs)

Document 1:

2.2. 표준/편집 테마
지도 빌더를 통해 제작된 지도를 표준, 편집 테마를 활용하여 앱의 레이아웃 및 스타일을 설정 하고 위젯들을 선택하여 새로운 앱을 생성할 수 있다.

2.2.1. 앱 만들기 화면
 테마 목록을 조회하고 테마를 선택하여 앱 편집 화면으로 이동한다.

2.2.2. 메인 설정
 메인 패널에서 사용할 사용자 지도를 선택한다. 

2.2.3. 레이아웃 설정
 타이틀 및 제어도구 위치를 설정한다.

2.2.4. 테마 설정
 헤더 및 푸터의 색상 및 타이틀 등 테마를 설정한다.

2.2.5. 위젯 설정
 웹앱의 위젯을 추가하고 저장한다.
 
헤더 영역 위젯 
앱 기본정보 
웹앱에 추가된 지도에 대한 상세정보를 확인할 수 있는 위젯
레이어 영역 위젯
 레이어 검색 레이어를 검색하고 추가할 수 있는 위젯
 레이어 스타일 레이어의 색상 및 라벨 등을 설정할 수 있는 위젯
 속성 레이어의 속성정보를 확인 할 수 있는 위젯
 속성필터 레이어의 속성으로 필터를 적용할 수 있는 위젯

상단 영역 위젯
 북마크 현재 지도의 위치를 저장할 수 있는 위젯
 오버뷰 오버뷰를 표출하는 위젯
 주소 검색 주소를 검색할 수 있는 위젯
 행정경계 표시 현재 지도의 행정경계 위치를 표시하고 다른 행정경계로 이동할 수 있는 위젯
 저장 지도를 PNG 또는 PDF로 저장할 수 있는 위젯
 인쇄 지도를 프린트 할 수 있는 위젯
 피쳐속성폼 위젯을 클릭한 후 지도상에 피쳐를 클릭하면 피쳐정보를 수정 할 수 있는 위젯
----------------------------------------------------------------------------------------------------
Document 2:

﻿맵픽이란
Mappick은 누구나 손쉽게 공간정보가 포함된 지도를 제작하여 업무에 적용 할 수 있는 SaaS(Software as a Service) GIS 서비스입니다. Mappick을 사용하면 인터넷을 통해 공개된 공공 데이터나, 본인이

- 압축기와 문서 변환기를 함께 연결하기
- DocumentCompressorPipeline을 사용하여 여러 압축기를 순차적으로 쉽게 결합할 수 있습니다.
- 압축기와 함께 BaseDocumentTransformers를 파이프라인에 추가할 수 있습니다. 이들은 문서 집합에 대해 단순히 변환을 수행하며, 맥락 압축은 수행하지 않습니다.
- 예를 들어 TextSplitters는 문서를 더 작은 조각으로 나누기 위해 문서 변환기로 사용할 수 있으며, EmbeddingsRedundantFilter는 문서 간 임베딩 유사성을 기반으로 중복 문서를 필터링하는 데 사용할 수 있습니다.
- 아래에서는 문서를 더 작은 청크로 나눈 다음 중복 문서를 제거하고 쿼리와의 관련성을 기준으로 필터링하여 압축기 파이프라인을 만듭니다.

In [12]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain_community.document_transformers import EmbeddingsRedundantFilter
from langchain_text_splitters import CharacterTextSplitter

splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=0, separator=". ")
redundant_filter = EmbeddingsRedundantFilter(embeddings=embeddings)
relevant_filter = EmbeddingsFilter(embeddings=embeddings, similarity_threshold=0.76)
pipeline_compressor = DocumentCompressorPipeline(
    transformers=[splitter, redundant_filter, relevant_filter]
)

In [13]:
compression_retriever = ContextualCompressionRetriever(
    base_compressor=pipeline_compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.invoke(
    query
)
pretty_print_docs(compressed_docs)

Created a chunk of size 334, which is longer than the specified 300


Document 1:

2.2. 표준/편집 테마
지도 빌더를 통해 제작된 지도를 표준, 편집 테마를 활용하여 앱의 레이아웃 및 스타일을 설정 하고 위젯들을 선택하여 새로운 앱을 생성할 수 있다.

2.2.1. 앱 만들기 화면
 테마 목록을 조회하고 테마를 선택하여 앱 편집 화면으로 이동한다.

2.2.2. 메인 설정
 메인 패널에서 사용할 사용자 지도를 선택한다. 

2.2.3. 레이아웃 설정
 타이틀 및 제어도구 위치를 설정한다.

2.2.4. 테마 설정
 헤더 및 푸터의 색상 및 타이틀 등 테마를 설정한다.

2.2.5
----------------------------------------------------------------------------------------------------
Document 2:

예 ) 밀도분석 : 점 레이어, 면적계산 : 면 레이어, 공간조인 분석 : 점/선/면 레이어
 
(지도빌더) 지오코딩 결과를 점 레이어와 면 레이어 둘 다 발행하려면 어떻게 하나요?
- 작업 알림창에서 지오코딩 이력을 삭제하지 않은 경우 [+] 버튼을 클릭하여 언제든지 레이어로 발행 할 수 있습니다.
 
(앱빌더) 표준 테마와 편집 테마는 어떻게 다르나요?
- 표준 테마, 편집 테마 대부분 동일한 위젯 도구 사용이 가능하지만 편집 테마에서만 속성 테이블의 편집 기능 사용이 가능합니다.
 
(앱빌더) 앱 빌더로 생성한 앱을 다른곳에서 사용이 가능한가요?
- 앱을 저장하고 미리보기 메뉴를 클릭 후 미리보기창에서 브라우저의 URL을 복사하여 타 시스템의 메뉴 등으로 연결하여 사용 가능합니다.
 
(개발자센터) 폐쇄망에서도 바로e맵, 브이월드 등 외부 연계 API를 사용 할 수 있나요?
- 폐쇄망의 경우 GeOn-API 서버가 외부망에 직접 연결이 가능하거나 별도의 내/외부망 연결이 가능한 Proxy 서버가 필요합니다.
 
(개발자센터) 지도 예제와 위젯 예제의 다른점은 무엇인가요?
- 지도 예제는 웹 환경에서 지도를 제어하는데 필요한 다양